<a href="https://colab.research.google.com/github/Beatriz-Yun/AIFFEL_LMS/blob/main/Exploration/%5BE-04%5D%EC%9E%91%EC%82%AC%EA%B0%80%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1) 데이터 전처리

## 1-1. 데이터 읽어오기

In [ ]:
import glob

txt_file_path = '/content/drive/MyDrive/Colab Notebooks/AIFFEL_LMS/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 한줄씩 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Ooh....... New York x2 Grew up in a town that is famous as a place of movie scenes', 'Noise is always loud, there are sirens all around and the streets are mean', "If I can make it here, I can make it anywhere, that's what they say"]


## 1-2. 데이터 정제

데이터 확인

In [ ]:
for idx, sentence in enumerate(raw_corpus):
    # 18만이 넘는 문장을 모두 확인할 수 없기 때문에
    # 적당히 500문장마다 텍스트를 확인해준다.
    if idx % 500 != 1:
      continue
    print(sentence)

    

Noise is always loud, there are sirens all around and the streets are mean
Will you ever know? You could buy me diamonds
Baby, I'm from [Chorus]
Say you want me, say you want me
That it will only get better
It may not be easy
With lovers and friends I still can recall
Love you, love you
Changing my life with a wave of her hand
Carry That Weight a long time
Despacito
And I'm fast enough to run across the sea (oh) 'Cause there's just no turnin' back
Cause this life's too long
We'll move out of the shelter
Uh-oh, I'm grateful for your existence
From here my dear
You smile I smile
We listened to The Smiths and The Violent Femmes
Sleep arm-in-arm in the stairwell
But not enough to slow you down I feel like the moon
Should I go back should I go back should I
'Cause you're the only thing I feel Someday
But to walk away, I have no capacity He walks away
Just to see that smile
You know that I'm no good Upstairs in bed with my ex-boy
She just stopped singing Ava was the morning, now she's gone
M

대괄호와 대괄호 안의 내용을 제거하고 싶어서 정규표현식을 `r"\[[^\]]+\]"`라고 표현하였다.<br>
대괄호 안에 포함되는 내용으로 `[^\]]`로 오른쪽 대괄호를 제외한 모든 문자를 지정했다.<br>
그 이유는 바깥범위 괄호 안에  내부의 괄호들이 포함되면 안되기 때문이다.<br><br>
올바른 정규표현식을 사용하였는지 확인하기 위해 findall함수의 결과로 저장된 match리스트의 원소를 출력해보았다.

In [ ]:
import re

# 가사가 아닌 텍스트가 얼마나 있는지 확인
pattern_list = ["instrumental", "chorus", "Chorus", r"\[[^\]]+\]"]
count = [0, 0, 0, 0]
count2 = 0
for idx, pat in enumerate(pattern_list):
  for sentence in raw_corpus:
    match = re.findall(pat, sentence)
    if match:
        #print(pat, str(len(match)), 'Found!')
        count[idx]+=1
    # else:
    #     print('Not Found!')
    if idx==3 and match!=[]:
      count2+=1
      print(match, end=' ')

print()
print(count)    # 각 패턴이 얼마나 발견되었는지 보여준다.
print(count2)   # count2는 대괄호안에 내용이 있는 패턴을 잘 찾아내었는지 count1[3]과 비교하기 위해 사용했다.

['[Chorus]'] ['[Chorus: x2]'] ['[Chorus]'] ['[Chorus: x2]'] ['[Chorus]'] ['[CHORUS]'] ['[CHORUS]'] ['[Chorus]'] ['[Chorus: x2]'] ['[Chorus]'] ['[Incomprehensible]'] ['[Chorus]'] ['[Chorus: x2]'] ['[Chorus]'] ['[Chorus: x2]'] ['[Chorus]'] ['[Chorus]'] ['[Chorus]'] ['[Chorus]'] ['[Chorus]'] ['[Chorus]'] ['[Chorus]'] ['[Chorus]'] ['[Verse 1]'] ['[Pre-Chorus]'] ['[Chorus]'] ['[Verse 2]'] ['[Pre-Chorus]'] ['[Chorus]'] ['[Verse 3]'] ['[Chorus]'] ['[Chorus]'] ['[Verse 1]'] ['[Chorus]'] ['[Verse 2]'] ['[Chorus]'] ['[Verse 1]'] ['[Chorus]'] ['[Verse 2]'] ['[Bridge]'] ['[Big Sean]'] ['[Chorus]'] ['[Verse 1]'] ['[Pre-Chorus]'] ['[Chorus]'] ['[Verse 2]'] ['[Pre-Chorus]'] ['[Chorus]'] ['[Verse 3]'] ['[Chorus]'] ['[Chorus]'] ['[Verse 1]'] ['[Chorus]'] ['[Verse 2]'] ['[Chorus]'] ['[Justin Bieber]'] ['[Justin Bieber]'] ['[Drake]'] ['[Justin Bieber]'] ['[Drake]'] ['[Justin Bieber]'] ['[both]'] ['[Drake]'] ['[Verse]'] ['[Pre-Chorus]'] ['[Chorus]'] ['[Verse]'] ['[Pre-Chorus]'] ['[Chorus]'] ['[Outro]'] ['

단어는 **공백**을 기준으로 나눌 것이다.<br>
그리고 I'm 또는 It'll, It's, 'Cause, We'll과 같이 '와 붙어있는 텍스트는 한 단어로 취급한다.

<br>

가사 텍스트 데이터를 훑어보고 처리해야겠다고 생각한 부분은 다음과 같다.

1. 양쪽 공백제거 및 모두 소문자로 변환
2. 문장부호 , ? . ! ¿가 있는 곳은 양옆에 공백을 추가해서 다른 단어와 분리해준다.
3. 대괄호[ ]와 그 안에 있는 내용을 공백으로 치환한다. (가사와 상관없는 텍스트가 들어있다. ex) chorus, 가수명 등)
4. 'instrumental'을 공백으로 치환한다. (반주중이라는 의미.)
5. 특수문자('포함)를 제거한다.
6. 공백이 여러 개 있는 부분은 공백 하나로 대체한다.
7. 다시 문장의 양쪽 공백을 제거한다.
8. 비어있지 않는 문장 앞에는 <start\>토큰, 뒤에는 <end\>토큰을 추가해준다.

In [ ]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                        # 1
    #print(sentence)
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 2
    #print(sentence)
    sentence = re.sub(r"\[[^\]]+\]", " ", sentence)            # 3
    #print(sentence)
    sentence = re.sub(r"instrumental", " ", sentence)          # 4
    #print(sentence)
    sentence = re.sub(r"[^a-zA-Z0-9?.!,¿']+", " ", sentence)  # 5
    #print(sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)                # 6
    #print(sentence)
    sentence = sentence.strip()                                # 7
    #print(sentence)

    if sentence:
      sentence = '<start> ' + sentence + ' <end>'              # 8
    return sentence

In [ ]:
# 문장이 어떻게 필터링되는지 확인 (print문 주석처리 이전)
sent = "This @_is ;;;sample        sentence. ten-year-old? [chorus] -instrumental-- I'll"
print('입력문장->',sent)
print(preprocess_sentence(sent))

입력문장-> This @_is ;;;sample        sentence. ten-year-old? [chorus] -instrumental-- I'll
this @_is ;;;sample        sentence. ten-year-old? [chorus] -instrumental-- i'll
this @_is ;;;sample        sentence .  ten-year-old ?  [chorus] -instrumental-- i'll
this @_is ;;;sample        sentence .  ten-year-old ?    -instrumental-- i'll
this @_is ;;;sample        sentence .  ten-year-old ?    - -- i'll
this is sample sentence . ten year old ? i'll
this is sample sentence . ten year old ? i'll
this is sample sentence . ten year old ? i'll
<start> this is sample sentence . ten year old ? i'll <end>


In [ ]:
# 여기에 전처리된 문장을 모은다.
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뛴다. (공백)
    if len(sentence) == 0: continue
    
    # 전처리함수에 문장을 입력 후 전처리된 문장을 corpus리스트에 추가.
    preprocessed_sentence = preprocess_sentence(sentence)   # (print문 주석처리 이후)
    
    # 토큰이 15개보다 많은 문장은 건너뛴다.
    if len(preprocessed_sentence.split(' '))>15: continue

    # 전처리함수의 결과로 None 또는 빈문자열일 수 있다.
    if preprocessed_sentence.split(' '):
      corpus.append(preprocessed_sentence)

# 문장 개수
print(len(corpus))

# 전처리된 문장 10개 확인
corpus[:10]

158821


["<start> baby , i'm from new york <end>",
 '<start> concrete jungle where dreams are made of <end>',
 "<start> there's nothing you can't do <end>",
 "<start> now you're in new york <end>",
 '<start> these streets will make you feel brand new <end>',
 '<start> big lights will inspire you <end>',
 "<start> baby , i'm from new york <end>",
 '<start> concrete jungle where dreams are made of <end>',
 "<start> there's nothing you can't do <end>",
 "<start> now you're in new york <end>"]

## 1-3. 토큰화

[tf.keras.preprocessing.sequence.pad_sequences함수 설명](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences)

In [ ]:
# 텐서플로우의 Tokenizer와 pad_sequences를 사용하여 토큰화한다.
import tensorflow as tf

# 15000단어를 기억할 수 있는 tokenizer
# 15000단어가 있는 사전에 포함되지 않는 단어는 '<unk>'로 바꾼다.
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)

    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus) 

    # 입력 데이터의 뒷부분을 0으로 패딩해준다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    print(tensor.shape)
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

(158821, 15)
[[   2   49    4 ...    0    0    0]
 [   2 2811 2083 ...    0    0    0]
 [   2  169  179 ...    0    0    0]
 ...
 [   2   28   17 ...    1    3    0]
 [   2  349   17 ...    3    0    0]
 [   2   37  130 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f0ad2ad6090>


In [ ]:
print(tensor[:3, :])

[[   2   49    4   20   67  153  723    3    0    0    0    0    0    0
     0]
 [   2 2811 2083   97  382   71  225   18    3    0    0    0    0    0
     0]
 [   2  169  179    7   86   44    3    0    0    0    0    0    0    0
     0]]


In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 50: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : the
6 : i
7 : you
8 : and
9 : a
10 : to
11 : me
12 : my
13 : it
14 : in
15 : that
16 : on
17 : your
18 : of
19 : .
20 : i'm
21 : like
22 : is
23 : all
24 : we
25 : be
26 : for
27 : up
28 : with
29 : so
30 : but
31 : just
32 : know
33 : love
34 : don't
35 : got
36 : no
37 : this
38 : get
39 : what
40 : they
41 : when
42 : ?
43 : oh
44 : do
45 : yeah
46 : she
47 : now
48 : if
49 : baby
50 : was


## 1-4. 훈련 데이터셋과 평가 데이터셋 분리

총 데이터의 20% 를 평가 데이터셋으로 사용한다.

In [ ]:
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]   # 처음~마지막-1
tgt_input = tensor[:, 1:]    # 처음+1~마지막

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=55)

데이터셋에서 랜덤으로 선택된 80%만큼의 훈련데이터셋의 크기이다.

In [ ]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (127056, 14)
Target Train: (127056, 14)


데이터셋 객체 생성

In [ ]:
BUFFER_SIZE = len(src_input)   # source sentence 수
BATCH_SIZE = 256               # 한 번 학습할 데이터 수
steps_per_epoch = len(src_input) // BATCH_SIZE

# VOCAB_SIZE: 벡터화한 단어 개수
# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)   # 데이터셋을 배치크기씩 나눈다.
                                                           # drop_remainder=True는 배치크기보다 작은 그룹을 버린다.
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# 2) 모델 구성

In [ ]:
class TextGenerator(tf.keras.Model):
  # embedding_size는 word vector의 차원수이다.
  # hidden_size는 LSTM레이어의 hidden state의 차원수이다.
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
# 데이터셋에서 데이터 한 배치만 불러온다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어보자.
model(src_sample)

<tf.Tensor: shape=(256, 14, 15001), dtype=float32, numpy=
array([[[-6.42370578e-05, -1.07015854e-04, -1.02676262e-04, ...,
          4.13059242e-05,  7.91815619e-05,  1.91425934e-05],
        [-1.22327721e-04, -3.48854112e-04, -1.16599804e-04, ...,
          2.28152610e-04,  9.19055019e-05,  2.68659351e-04],
        [-3.91875888e-04, -7.53951259e-04, -3.39158250e-05, ...,
          3.74744122e-04,  4.09074331e-04,  2.37443950e-04],
        ...,
        [ 1.23066886e-03,  2.74594495e-04, -6.68036751e-04, ...,
          4.13758942e-04, -2.88598298e-04,  6.43533422e-04],
        [ 1.45023246e-03,  3.42591055e-04, -7.21704739e-04, ...,
          5.29327022e-04, -4.84824472e-04,  6.97431562e-04],
        [ 1.64344627e-03,  3.79976380e-04, -7.79801980e-04, ...,
          6.50570262e-04, -6.61915401e-04,  7.21280347e-04]],

       [[-6.42370578e-05, -1.07015854e-04, -1.02676262e-04, ...,
          4.13059242e-05,  7.91815619e-05,  1.91425934e-05],
        [-3.05340596e-04, -4.96728171e-04, -7

In [ ]:
model.summary()

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  3840256   
_________________________________________________________________
lstm_6 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_7 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_3 (Dense)              multiple                  15376025  
Total params: 32,855,961
Trainable params: 32,855,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tf.test.is_gpu_available()

True

In [ ]:
optimizer = tf.keras.optimizers.Adam()
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
620/620 [==============================] - 229s 365ms/step - loss: 3.4573
Epoch 2/30
620/620 [==============================] - 225s 363ms/step - loss: 3.0306
Epoch 3/30
620/620 [==============================] - 225s 363ms/step - loss: 2.8689
Epoch 4/30
620/620 [==============================] - 226s 363ms/step - loss: 2.7410
Epoch 5/30
620/620 [==============================] - 225s 363ms/step - loss: 2.6322
Epoch 6/30
620/620 [==============================] - 225s 363ms/step - loss: 2.5352
Epoch 7/30
620/620 [==============================] - 225s 363ms/step - loss: 2.4472
Epoch 8/30
620/620 [==============================] - 224s 362ms/step - loss: 2.3644
Epoch 9/30
620/620 [==============================] - 224s 361ms/step - loss: 2.2859
Epoch 10/30
620/620 [==============================] - 225s 362ms/step - loss: 2.2125
Epoch 11/30
620/620 [==============================] - 227s 366ms/step - loss: 2.1426
Epoch 12/30
620/620 [==============================] - 227s 365

In [ ]:
model.save('lyrics_model2')

INFO:tensorflow:Assets written to: lyrics_model2/assets


INFO:tensorflow:Assets written to: lyrics_model2/assets


In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you , <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> kiss you", max_len=100)

'<start> kiss you taste you all night <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=100)

'<start> love is a beautiful thing <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> I", max_len=100)

"<start> i know that we gon' be alright <end> "

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> American", max_len=100)

'<start> american idol never seen these dreams <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> my", max_len=100)

"<start> my name is prince and i am funky you can't stop prince <end> "

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> babe", max_len=100)

'<start> babe , i know that you are free <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> why", max_len=100)

'<start> why do you love me <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> what are you", max_len=100)

'<start> what are you , what are you waiting for ? <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> kiss you", max_len=100)

'<start> kiss you on ya neck and tell ya everything is great <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> kiss you", max_len=100)

'<start> kiss you on ya neck and tell ya everything is great <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> are you", max_len=100)

'<start> are you ready <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> summer night", max_len=100)

'<start> summer night , my body s callin for you <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> over the", max_len=100)

'<start> over the water to charley <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> cat", max_len=100)

'<start> cat , we need u 2 rap <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> will you", max_len=100)

'<start> will you still love me tomorrow <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> taxi", max_len=100)

'<start> taxi ! taxi ! <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> during the", max_len=100)

'<start> during the days you helped me bag up my nickels <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=100)

'<start> love is a feeling <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> feeling", max_len=100)

'<start> feeling the catastrophe , but she knows she can fly away <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> why", max_len=100)

'<start> why you wanna get with me <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> I", max_len=100)

'<start> i m a bad man <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> loving", max_len=100)

'<start> loving you is more important <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> American", max_len=100)

'<start> american , american oxygen american oxygen <end> '

In [ ]:
# from google.colab import files
# files.download('lyrics_model1')  # from colab to browser download

# from keras.models import load_model
# model2 = load_model('lyrics_model1')

In [ ]:
generate_text(model2, tokenizer, init_sentence="<start> I", max_len=100)

ValueError: ignored

In [ ]:
model2 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
model2.load_weights('lyrics_weight2')

In [ ]:
generate_text(model2, tokenizer, init_sentence="<start> I", max_len=100)

'<start> i m a bad man <end> '